# notre appli

In [2]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, dcc, html, Dash,dcc,html,callback
import pandas as pd 
import plotly.express as px
import Functions_dash as Fun
import content as cont
import pages as pag

In [4]:
# link fontawesome to get the chevron icons
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME],
    suppress_callback_exceptions=True # If components are constructed by callbacks
)

# Bar noir en haut de page
navbar = dbc.NavbarSimple(
    children=[
        dbc.Button("Menu", outline=True, color="secondary", className="mr-1", id="btn_sidebar"), # Bouton pour le menu
    ],
    brand="",
    brand_href="#",
    color="dark",
    dark=True,
    fluid=True,
    links_left= True,
)

# Menus déplient 1
submenu_1 = [
    html.Li(
        # use Row and Col components to position the chevrons
        dbc.Row(
            [
                dbc.Col("Menu 1"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-1",
    ),
    # we use the Collapse component to hide and reveal the navigation links
    dbc.Collapse(
        [
            dbc.NavLink("Page principle", href="/page-main"), # Page d'acceuil
            dbc.NavLink("Carte des accidents en France", href="/page-map"), # Map page
        ],
        id="submenu-1-collapse",
    ),
]

# Menu déplient 2
submenu_2 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col("Menu 2"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-2",
    ),
    dbc.Collapse(
        [
            dbc.NavLink("Page 2.1", href="/page-2/1"),
            dbc.NavLink("Page 2.2", href="/page-2/2"),
        ],
        id="submenu-2-collapse",
    ),
]

# Div contenent le menu déplient
sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "A sidebar with collapsible navigation links", className="lead"
        ),
        dbc.Nav(submenu_1 + submenu_2, vertical=True),
    ],
    style=cont.SIDEBAR_STYLE,
    id="sidebar",
)

# Div racine de la page
app.layout = html.Div(
    [
        dcc.Store(id='side_click'),
        dcc.Location(id="url"),
        navbar,
        sidebar,
        html.Div(id="page-content", style=cont.CONTENT_STYLE), # Elément qui contien la page principale
    ],
)

# Fonction qui appel toute les fonction callback qui sont dans le fichier Fonction_dash.py
Fun.get_callbacks(app)



if __name__ == "__main__":
    app.run_server(debug=True,jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


## Temp storage
### Number bar children

In [ ]:
dbc.Row(
                        [
                            dbc.Col(cont.nb_total, width=3),
                            dbc.Col(cont.nb_mort, width=3),
                            dbc.Col(cont.nb_hospital, width=3),
                            dbc.Col("Info 4", width=3),
                        ],
                    ),